In [2]:
%load_ext autoreload
%autoreload 2

import sys, os

# Go up one directory from `b/` to project root
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

In [2]:
import logging
from neurosurfer.models.chat_models.transformers import TransformersModel
from neurosurfer import config 
import torch
torch.cuda.empty_cache()

DEFAULT_TRANSFORMERS_MODEL_PARAMS = dict({
    "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
    "max_seq_length": 16_000,
    "load_in_4bit": True,
    "enable_thinking": False,  # main_gpu interpretation
    "verbose": False
})

LOGGER = logging.getLogger()
LLM = TransformersModel(
    **DEFAULT_TRANSFORMERS_MODEL_PARAMS,
    stop_words=["Observation:"],
    logger = logging.getLogger(),
)

╔══════════════════════════════════════════════════════════════════╗
║                                                                  ║
║ ▓▓▓▓▓   ▓▓▓▓                                  ▓▓▓                ║
║  ▓▓ ▓▓   ▓▓  ▓▓▓▓ ▓  ▓ ▓ ▓ ▓▓▓▓ ▓▓▓ ▓  ▓ ▓ ▓  ▓   ▓▓▓▓ ▓ ▓       ║
║  ▓▓  ▓▓  ▓▓  ▓▁▁▓ ▓  ▓ ▓▓▏ ▓  ▓ ▓▁  ▓  ▓ ▓▓▏ ▓▓▓  ▓▁▁▓ ▓▓        ║
║  ▓▓   ▓▓ ▓▓  ▓    ▓  ▓ ▓   ▓  ▓   ▓ ▓  ▓ ▓    ▓   ▓    ▓         ║
║ ▓▓▓▓   ▓▓▓▓▓ ▓▓▓▓ ▓▓▓▓ ▓   ▓▓▓▓ ▓▓▓ ▓▓▓▓ ▓    ▓   ▓▓▓▓ ▓         ║
║ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ║
║ Orchestrate Agents - RAG - SQL Tools - Multi-LLM - FastAPI Ready ║
║ Faster builds, clearer flows, production-first                   ║
║                                                                  ║
║ Version: 0.1.3 | Python: 3.11.13                                 ║
║ OS: Linux 6.14.0-35-generic (x86_64)                             ║
║ Torch: 2.7.1+cu126   CUDA: yes (12.6)                            ║
║ MPS: no (built: False)          

/home/nomi/anaconda3/envs/LLMs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO     | 2025-11-20 10:03:01 | config.py:<module> | PyTorch version 2.7.1+cu126 available.


/home/nomi/anaconda3/envs/LLMs/lib/python3.11/importlib/__init__.py:126: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  return _bootstrap._gcd_import(name[level:], package, level)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-20 10:03:02 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO     | 2025-11-20 10:03:04 | transformers.py:init_model | Initializing Transformers model.
WARNING  | 2025-11-20 10:03:04 | transformers.py:init_model | Model is already quantized. Ignoring load_in_4bit=True.
INFO     | 2025-11-20 10:03:05 | modeling.py:get_balanced_memory | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.86s/it]

INFO     | 2025-11-20 10:03:14 | transformers.py:init_model | Transformers model initialized successfully.


In [3]:
# streaming response example
from IPython.display import display, Markdown, clear_output

system_prompt = "You are a joker."
user_prompt = """Tell me a short and light-hearted joke."""

stream_response = LLM.ask(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    stream=True
)

md_display = display(Markdown(""), display_id=True)
for chunk in stream_response:
    chunk = chunk.choices[0].delta.content or ""
    print(chunk, flush=True, end="")

Why don't skeletons fight each other?  
Because they don't have the *guts*! 😄

### Agent Test

In [4]:
# agent normal response
from neurosurfer.agents import Agent, AgentConfig
# from neurosurfer.tracing import RichTracer
from pydantic import BaseModel

agent_config = AgentConfig(
    strict_tool_call=True,
    return_stream_by_default=True,
    temperature=0.7,
    max_new_tokens=4096,
)
agent = Agent(llm=LLM, config=agent_config, log_traces=True)

# normal response
print("Normal Response:")
agent_response = agent.run(user_prompt="What is AI (one line)?", stream=False)
# print(agent_response.response)

# # streaming response
# print("\n\nStreaming Response:")
# for c in agent.run(user_prompt="What are top 3 applications of AI (one line)?").response:
#     print(c, flush=True, end="")


Normal Response:
🧠 Thinking...

[main_agent] Tracing Start!
 ▶ [1][step.agent] agent_id='main_agent' label='agent.run'
     ▶ [2][step.llm.call] agent_id='main_agent' label='agent.free_text_call'
     ◀ [2][step.llm.call] agent_id='main_agent' label='agent.free_text_call' took 0.876s; error=False
 ◀ [1][step.agent] agent_id='main_agent' label='agent.run' took 0.877s; error=False
[main_agent] Tracing End!

Final response:
AI, or Artificial Intelligence, is the simulation of human intelligence in machines that are programmed to think, learn, and perform tasks typically requiring human cognition.


**Structured Response**

In [5]:
# Structured Response examples
class AIApplication(BaseModel):
    title: str
    description: str

class AI(BaseModel):
    definition: str
    history: str
    modern_frameworks: str
    applications: list[AIApplication]

user_query = "What is AI and list 3 of its top application, and 3 concerns."
agent_response = agent.run(user_prompt=user_query, output_schema=AI)

print("\n\nStructured Response:")
print(agent_response.response.json_obj)  

WARNING  | 2025-11-20 10:03:24 | agent.py:run     | `output_schema` provided with `stream=True`; forcing non-streaming structured output.
🧠 Thinking...

[main_agent] Tracing Start!
 ▶ [1][step.agent] agent_id='main_agent' label='agent.run'
     ▶ [2][step.llm.call] agent_id='main_agent' label='agent.structured_call.first_pass'
     ◀ [2][step.llm.call] agent_id='main_agent' label='agent.structured_call.first_pass' took 5.446s; error=False
 ◀ [1][step.agent] agent_id='main_agent' label='agent.run' took 5.447s; error=False
[main_agent] Tracing End!



Structured Response:
{
  "definition": "Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and language understanding.",
  "history": "The concept of AI was first introduced in the 1950s. Early developments included the creation of the first AI programs and the establishment of AI as a formal academ

## RAG wiring so the agent “understands” the Neurosurf codebase

You’ll ingest the repo once, then run a retriever to answer code questions. The Planner can call the retriever first to form a precise implementation plan.

### FileReader and Chunker Test

In [10]:
from pathlib import Path
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder
from neurosurfer.agents.rag.chunker import Chunker
from neurosurfer.agents.rag.filereader import FileReader
from neurosurfer.agents.rag import RAGAgent, RAGAgentConfig, RAGIngestorConfig

chunker = Chunker()
file_reader = FileReader()
embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")

rag_agent = RAGAgent(
    llm=LLM,
    embedder=embedder,
    file_reader=file_reader,
    chunker=chunker,
    config=RAGAgentConfig(
        top_k=5,
        fixed_max_new_tokens=1024,
        clear_collection_on_init=True
    ),
    ingestor_config=RAGIngestorConfig(
        batch_size=64,
        max_workers=4,
        deduplicate=True,
        normalize_embeddings=True,
        default_metadata=None,
        tmp_dir="./rag-storage",
    ),
)

dir_path = "../neurosurfer"
summary = rag_agent.ingest(sources=dir_path)

summary

INFO     | 2025-11-20 10:08:56 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-11-20 10:08:56 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
WARNING  | 2025-11-20 10:08:56 | agent.py:__init__ | No vectorstore provided to RAGAgent, using default ChromaVectorStore. Initializing default collection `neurosurfer-rag-agent`
[Init] ChromaVectorStore initialized with collection: neurosurfer-rag-agent


Batches: 100%|██████████| 2/2 [00:00<00:00, 16.87it/s]


{'status': 'ok',
 'sources': 141,
 'chunks': 889,
 'unique_chunks': 889,
 'added': 889,
 'finished_at': 1763618939.3266816,
 'accepted_sources': 1,
 'total_docs_in_collection': 889}

In [56]:

retrival_results = rag_agent.retrieve(user_query="Explain how to use the graph agent and its configurations?", retrieval_mode="smart")
print(retrival_results.context)


------------------ RetrievalPlan(mode='smart', scope='medium', top_k=15, neighbor_hops=1, notes=None, extra={'scope': 'medium', 'top_k': 15, 'neighbor_hops': 1})


Batches: 100%|██████████| 1/1 [00:00<00:00, 58.28it/s]

Source: e3cb346a127a70f3:a1074d30
NODE_TOOLS:
{tools}
GRAPH_INPUTS (as JSON-ish):
{graph_inputs}
DEPENDENCY_RESULTS (node_id -> result):
{dependency_results}
PREVIOUS_RESULT (may be empty if none):
{prev_txt}
Compose the next user_prompt string that this node's agent should receive.
Return ONLY that prompt text.
"""
DEFAULT_NODE_SYSTEM_TEMPLATE = """You are a specialized agent in a larger workflow.
Your role:
- PURPOSE: {purpose}
- GOAL: {goal}
- EXPECTED_RESULT: {expected_result}
General behaviour:
- Be precise and concise unless the task requires extended output.
- Use clear structure (headings/bullets) when helpful.
- If you are calling tools, interpret their outputs carefully and explain your reasoning.
"""

---

Source: 0fb16151b97daeb7:d679f9d7
                node_config.retry.max_route_retries = p.retries
            if p.max_new_tokens is not None:
                node_config.max_new_tokens = p.max_new_tokens
            if p.temperature is not None:
                node_confi

**Server RAG Orchestrator**

In [62]:
# scripts/index_repo_for_rag.py
from pathlib import Path
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder
from neurosurfer.server.services.rag.orchestrator import RAGOrchestrator

try:
    from neurosurfer.server.db.db import init_db
    init_db()
except Exception as _e:
    print('DB init warning:', _e)
    
embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
# vs = ChromaVectorStore(collection_name="neurosurf-repo")
RAG = RAGOrchestrator(
    embedder=embedder,
    gate_llm=LLM,
    top_k=10,
    verbose=True,
    logger=LOGGER,
)

INFO     | 2025-11-20 10:30:58 | db.py:init_db    | Database initialized successfully in sqlite:////home/nomi/.cache/Neurosurfer/app-storage/app.db


INFO     | 2025-11-20 10:30:59 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-11-20 10:30:59 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
WARNING  | 2025-11-20 10:30:59 | agent.py:__init__ | No vectorstore provided to RAGAgent, using default ChromaVectorStore. Initializing default collection `neurosurfer-rag-agent`
[Init] ChromaVectorStore initialized with collection: neurosurfer-rag-agent
INFO     | 2025-11-20 10:30:59 | orchestrator.py:_reset_db | Number of users left: 1
INFO     | 2025-11-20 10:30:59 | orchestrator.py:_reset_db | Number of files left: 0
INFO     | 2025-11-20 10:30:59 | orchestrator.py:_reset_db | Number of threads left: 0
INFO     | 2025-11-20 10:30:59 | orchestrator.py:_reset_db | Number of messages left: 0
INFO     | 2025-11-20 10:30:59 | orchestrator.py:_reset_db | Number of documents left: 0


In [ ]:
from typing import List, Dict, Any
import base64
import os

actor_id = 1
thread_id = 1
user_query = "Explain the weather tool and write its summary.?"

file_path = "weather_tool.py"

# os.path.exists(file_path)

files: List[Dict[str, Any]] = [{
    "name": file_path,
    "content": base64.b64encode(open(file_path, "rb").read()).decode("utf-8"),
    "type": "text/x-python",
}]

# reset db
RAG._reset_db()

rag_res = RAG.apply(
    actor_id=actor_id,
    thread_id=thread_id,
    user_query=user_query,
    files=files,
)
user_query = rag_res.augmented_query
if rag_res.used:
    LOGGER.info(f"[RAG] used context (top_sim={rag_res.meta.get('top_similarity', 0):.3f})")
else:
    LOGGER.info(f"[RAG] no context (reason={rag_res.meta.get('reason')})")

In [3]:
# for chunk in LLM.ask(
#     user_prompt=rag_res.augmented_query,
#     system_prompt="You are a helpful assistant. Provide your answers in a clear and concise manner.",
#     chat_history=[],
#     stream=True,
# ):
#     chunk = chunk.choices[0].delta.content or ""
#     print(chunk, end="", flush=True)

## Server APP

In [6]:
import logging
from neurosurfer.models.chat_models.transformers import TransformersModel
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder
from neurosurfer.config import config


LOGGER = logging.getLogger("neurosurfer")
try:
    import torch
    LOGGER.info(f"GPU Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        LOGGER.info(f"GPU Name: {torch.cuda.get_device_name(0)}")
except Exception:
    LOGGER.warning("Torch not found...")
    
LLM = TransformersModel(
    # model_name="unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    model_name="/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
    max_seq_length=config.base_model.max_seq_length,
    load_in_4bit=config.base_model.load_in_4bit,
    enable_thinking=config.base_model.enable_thinking,
    stop_words=config.base_model.stop_words or [],
    logger=LOGGER,
)
EMBEDDER_MODEL = SentenceTransformerEmbedder(
    model_name="intfloat/e5-large-v2", 
    logger=LOGGER
)

INFO     | 2025-11-21 08:57:37 | 194622058.py:<module> | GPU Available: True
INFO     | 2025-11-21 08:57:37 | 194622058.py:<module> | GPU Name: NVIDIA GeForce RTX 3080 Ti
INFO     | 2025-11-21 08:57:37 | transformers.py:init_model | Initializing Transformers model.
WARNING  | 2025-11-21 08:57:37 | transformers.py:init_model | Model is already quantized. Ignoring load_in_4bit=True.
INFO     | 2025-11-21 08:57:37 | modeling.py:get_balanced_memory | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.56s/it]

INFO     | 2025-11-21 08:57:43 | transformers.py:init_model | Transformers model initialized successfully.


INFO     | 2025-11-21 08:57:45 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-11-21 08:57:45 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.


In [ ]:
from typing import List, Generator
import os, shutil, logging
from neurosurfer.models.chat_models import BaseChatModel as BaseChatModel
from neurosurfer.server.app import NeurosurferApp
from neurosurfer.server.schemas import ChatCompletionRequest, ChatCompletionResponse, ChatCompletionChunk
from neurosurfer.server.runtime import RequestContext
from neurosurfer.server.services.rag import RAGOrchestrator

from neurosurfer.config import config
logging.basicConfig(level=config.app.logs_level.upper())


# Create app instance
ns = NeurosurferApp(
    app_name=config.app.app_name,
    api_keys=[],
    enable_docs=config.app.enable_docs,
    cors_origins=config.app.cors_origins,
    host=config.app.host_ip,
    port=config.app.host_port,
    reload=config.app.reload,
    log_level=config.app.logs_level,
    workers=config.app.workers
)


LOGGER: logging.Logger = None
RAG: RAGOrchestrator | None = None

@ns.on_startup
async def load_model():
    global EMBEDDER_MODEL, LOGGER, LLM, RAG
    # registered models are visible in the UI. You must register a model for it to be available in the UI.
    ns.model_registry.add(
        llm=LLM,
        family="llama",
        provider="Unsloth",
        description="Proxy to Llama"
    )
    RAG = RAGOrchestrator(
        embedder=EMBEDDER_MODEL,
        gate_llm=LLM,
        top_k=10,
        verbose=True,
        logger=LOGGER,
    )
    # Warmup
    joke = LLM.ask(user_prompt="Say hi!", system_prompt=config.base_model.system_prompt, stream=False)
    LOGGER.info(f"LLM ready: {joke.choices[0].message.content}")

@ns.on_shutdown
def cleanup():
    """Clean up temporary files and directories on application shutdown."""
    pass

@ns.chat()
def handler(request: ChatCompletionRequest, ctx: RequestContext) -> ChatCompletionResponse | Generator[ChatCompletionChunk, None, None]:
    global LLM, RAG
    # Resolve actor/thread ids (thread id is part of JSON now)
    user_id = ctx.user_id
    thread_id = request.thread_id
    has_files_message = request.files is not None

    # Prepare inputs
    user_msgs: List[str] = [m["content"] for m in request.messages if m["role"] == "user"]
    system_msgs = [m["content"] for m in request.messages if m["role"] == "system"]
    system_prompt = system_msgs[0] if system_msgs else config.base_model.system_prompt    # First system message or default
    user_query = user_msgs[-1] if user_msgs else ""    # Last user message

    # Minimal chat history excluding system messages, max 10
    num_recent = 10
    conversation_messages = [msg for msg in request.messages if msg["role"] != "system"]
    chat_history = conversation_messages[-num_recent:-1]

    temperature = request.temperature if (request.temperature and 2 > request.temperature > 0) else config.base_model.temperature
    max_tokens = request.max_tokens if (request.max_tokens and request.max_tokens > 512) else config.base_model.max_new_tokens
    kwargs = {"temperature": temperature, "max_new_tokens": max_tokens}

    if RAG and thread_id is not None:
        rag_res = RAG.apply(
            user_id=user_id,
            thread_id=thread_id,
            user_query=user_query,
            has_files_message=has_files_message,
        )
        user_query = rag_res.augmented_query
        if rag_res.used:
            LOGGER.info(f"[RAG] used context (top_sim={rag_res.meta.get('top_similarity', 0):.3f})")
        else:
            LOGGER.info(f"[RAG] no context (reason={rag_res.meta.get('reason')})")

    # Model call (stream or non-stream handled by router)
    return LLM.ask(
        user_prompt=user_query,
        system_prompt=system_prompt,
        chat_history=chat_history,
        stream=request.stream,
        **kwargs
    )

@ns.stop_generation()
def stop_handler():
    print("Stopping generation...")
    global LLM
    LLM.stop_generation()

def create_app():
    return ns.app

def main():
    ns.run()

main()


INFO     | 2025-11-21 08:58:13 | db.py:init_db    | Database initialized successfully in sqlite:////home/nomi/.cache/Neurosurfer/app-storage/app.db
ERROR    | 2025-11-21 08:58:13 | app.py:run       | Failed to run app: asyncio.run() cannot be called from a running event loop


Traceback (most recent call last):
  File "/home/nomi/workspace/neurosurfer/neurosurfer/server/app.py", line 582, in run
    uvicorn.run(
  File "/home/nomi/anaconda3/envs/LLMs/lib/python3.11/site-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/home/nomi/anaconda3/envs/LLMs/lib/python3.11/site-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nomi/anaconda3/envs/LLMs/lib/python3.11/asyncio/runners.py", line 186, in run
    raise RuntimeError(
RuntimeError: asyncio.run() cannot be called from a running event loop
/home/nomi/workspace/neurosurfer/neurosurfer/server/app.py:593: RuntimeWarning: coroutine 'Server.serve' was never awaited
  self.logger.error(f"Failed to run app: {e}")


In [ ]:
# searches.results
# # print(searches.results["rag_content"])
# print(searches.results["llm_summary"])


{'query': 'Importance of sleep in health.',
 'summary': "Top 3 results out of ~988,000,000 results for: 'Importance of sleep in health.'\n1. How Sleep Works - Why Is Sleep Important? - NHLBI - NIH — https://www.nhlbi.nih.gov/health/sleep/why-sleep-important\n2. Better sleep: Why it's important for your health and tips to ... — https://health.ucdavis.edu/blog/cultivating-health/better-sleep-why-its-important-for-your-health-and-tips-to-sleep-soundly/2023/03\n3. Sleep is essential to health - Journal of Clinical Sleep Medicine — https://jcsm.aasm.org/doi/10.5664/jcsm.9476",
 'provider': 'serpapi',
 'elapsed_ms': 3004,
 'rag_content': 'Source: 355e311e80b4be26:cc35f490\nlth Sleep has become a precious commodity – we love it and need it, but rarely get enough of it. Busy schedules, kids, anxiety and technology can all get in the way of a good night sleep. Getting enough sleep can play an important role in your weight, emotional wellbeing, blood pressure, diabetes, mental and physical perfo

**GrpahAgent from YML file**

In [95]:
from neurosurfer.models.chat_models.base import BaseChatModel
from neurosurfer.agents.graph import GraphAgent, ManagerConfig

graph_agent = GraphAgent(
    llm=LLM,
    graph_yaml="blog_workflow.yml",
    toolkit=toolkit,
    manager_config=ManagerConfig(
        temperature=0.5,
        max_new_tokens=4096,
    ),
    manager_llm=LLM,
    log_traces=True
)

# Run workflow
graph_inputs = {
    "topic_title": "The Missing Middle Layer: Why LLM Systems Need Tool Routers, Not Bigger Models",
    "query": "Compose a 2000-2500 word blog on why tool-routing layers matter more than scaling LLM size, covering practical design patterns, examples, and tradeoffs.",
    "audience": "Intermediate ML engineers",
    "tone": "Practical and slightly opinionated",
}

results = graph_agent.run(inputs=graph_inputs)
# result = await run_async(executor.run(inputs=graph_inputs))


WARNING  | 2025-11-18 14:58:05 | utils.py:normalize_and_validate_graph_inputs | Ignoring extra inputs not declared in graph spec: audience, tone
INFO     | 2025-11-18 14:58:05 | toolkit.py:register_tool | Registered tool: web_search

[manager] Tracing Start!
 ▶ [1][step.llm.call] agent_id='manager' label='manager.compose_user_prompt'
 ◀ [1][step.llm.call] agent_id='manager' label='manager.compose_user_prompt' took 2.249s; error=False
[manager] Tracing End!

🧠 Thinking...

[research] Tracing Start!
 ▶ [1][step.agent] agent_id='research' label='agent.run'
     ▶ [2][step.llm.call] agent_id='research' label='agent.route_and_call.router_llm_call'
        INFO: Selected tool: web_search
        INFO: Raw inputs: {'query': 'The Missing Middle Layer: Why LLM Systems Need Tool Routers, Not Bigger Models'}
     ◀ [2][step.llm.call] agent_id='research' label='agent.route_and_call.router_llm_call' took 1.079s; error=False
     ▶ [3][step.tool.execute] agent_id='research' label='agent.route_and_ca

Batches: 100%|██████████| 1/1 [00:00<00:00, 53.60it/s]

INFO     | 2025-11-18 14:58:09 | agent.py:retrieve | [RAGRetriever] Retrieved 10 chunks
        INFO: Tool 'web_search' Tool Return: {'query': 'The Missing Middle Layer: Why LLM Systems Need Tool Routers, Not Bigger Models', 'summary...
     ◀ [3][step.tool.execute] agent_id='research' label='agent.route_and_call.tool_execute' took 0.933s; error=False
 ◀ [1][step.agent] agent_id='research' label='agent.run' took 2.014s; error=False
[research] Tracing End!


[manager] Tracing Start!
 ▶ [2][step.llm.call] agent_id='manager' label='manager.compose_user_prompt'


 ◀ [2][step.llm.call] agent_id='manager' label='manager.compose_user_prompt' took 0.790s; error=True
Error executing node outline: CUDA out of memory. Tried to allocate 158.00 MiB. 
GPU 0 has a total capacity of 11.61 GiB of which 120.00 MiB is free. Including 
non-PyTorch memory, this process has 10.70 GiB memory in use. Of the allocated 
memory 9.91 GiB is allocated by PyTorch, and 484.27 MiB is reserved by PyTorch 
but unallocated. If reserved but unallocated memory is large try setting 
PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See 
documentation for Memory Management  
(https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Returning partial results from executed nodes...
INFO     | 2025-11-18 14:58:10 | export.py:export_single_node | Exported node research output to exports/research_20251118_145807.json


In [52]:
import json

with open("graph_agent_results.json", "w") as writer:
    json.dump(results.model_dump(), writer)

In [53]:

print(results["results"]["review"].raw_output)

# Review of Draft: "Understanding Tool-Augmented LLM Agents: Architecture, Workflow, and Best Practices"

---

## ✅ **Strengths**

- **Comprehensive Overview**: The draft provides a well-structured overview of tool-augmented LLM agents, covering key components like architecture, workflow, and best practices.
- **Clear Terminology**: The use of consistent terminology (e.g., "tool-augmented agents", "agent loop") helps maintain clarity.
- **Practical Focus**: The inclusion of best practices and workflow diagrams adds practical value for developers and researchers.
- **Up-to-Date References**: The draft references recent advancements in LLM agent systems, such as the use of retrieval-augmented generation (RAG) and tool integration.

---

## ❌ **Issues and Concerns**

### 1. **Technical Inaccuracies**

- **Overgeneralization of Tool Integration**: The draft refers to "tools" in a broad sense, but does not clearly distinguish between different types of tools (e.g., API-based, database, or e

In [ ]:
from __future__ import annotations
from typing import List, Optional
from pydantic import BaseModel, Field, field_validator
from dataclasses import dataclass, field
from typing import Dict

@dataclass
class RouterRetryPolicy:
    """Retry tuning for routing + tool execution."""
    max_route_retries: int = 2
    max_tool_retries: int = 1
    backoff_sec: float = 0.7  # linear backoff

@dataclass
class AgentConfig:
    """
    Top-level configuration for the Agent.
    """
    # Routing:
    allow_input_pruning: bool = True    # drop unknown inputs not in ToolSpec
    repair_with_llm: bool = True        # ask LLM to repair invalid routing/inputs
    strict_tool_call: bool = False      # router must output JSON; else can answer in plain text
    # synonyms: Dict[str, Dict[str, str]] = field(default_factory=dict)  # field -> {from: to}

    # LLM defaults:
    temperature: float = 0.7
    max_new_tokens: int = 512
    return_stream_by_default: bool = False

    # Retries:
    retry: RouterRetryPolicy = field(default_factory=RouterRetryPolicy)

    # Structured-output options:
    strict_json: bool = True                  # enforce RFC 8259 JSON
    max_repair_attempts: int = 1              # for malformed JSON repairs



class NodeBudget(BaseChatModel):
    """
    Budget / LLM-related overrides per node.

    These map directly to AgentConfig fields:
        - temperature      -> AgentConfig.temperature
        - max_new_tokens   -> AgentConfig.max_new_tokens
        - return_stream_by_default -> AgentConfig.return_stream_by_default
    """

    max_new_tokens: Optional[int] = Field(
        default=None,
        description="Override AgentConfig.max_new_tokens for this node only.",
    )
    temperature: Optional[float] = Field(
        default=None,
        description="Override AgentConfig.temperature for this node only.",
    )
    return_stream_by_default: Optional[bool] = Field(
        default=None,
        description="Override AgentConfig.return_stream_by_default for this node only.",
    )


class NodePolicy(BaseChatModel):
    """
    Per-node policy that can override some AgentConfig settings and add
    node-level execution constraints (e.g., timeout).

    YAML example:

        nodes:
          - id: research
            policy:
              retries: 1
              timeout_s: 30
              budget:
                max_new_tokens: 180
                temperature: 0.2
              allow_input_pruning: false
              repair_with_llm: true
              strict_tool_call: true
    """

    retries: Optional[int] = Field(
        default=None,
        description="Override AgentConfig.retry.max_route_retries for this node.",
    )
    timeout_s: Optional[int] = Field(
        default=None,
        description=(
            "Soft timeout for this node in seconds. Execution isn't forcibly "
            "cancelled but the node will be marked as errored if exceeded."
        ),
    )
    budget: Optional[NodeBudget] = None

    # Direct AgentConfig-like overrides
    allow_input_pruning: Optional[bool] = None
    repair_with_llm: Optional[bool] = None
    strict_tool_call: Optional[bool] = None
    strict_json: Optional[bool] = None
    max_repair_attempts: Optional[int] = None

    class Config:
        extra = "ignore"  # ignore unknown keys under 'policy'

c = AgentConfig()

p = NodePolicy(budget=NodeBudget(temperature=1.2))

print(c)
print(p)


AgentConfig(allow_input_pruning=True, repair_with_llm=True, strict_tool_call=False, temperature=0.7, max_new_tokens=512, return_stream_by_default=False, retry=RouterRetryPolicy(max_route_retries=2, max_tool_retries=1, backoff_sec=0.7), strict_json=True, max_repair_attempts=1)
retries=None timeout_s=None budget=NodeBudget(max_new_tokens=None, temperature=1.2, return_stream_by_default=None) allow_input_pruning=None repair_with_llm=None strict_tool_call=None strict_json=None max_repair_attempts=None


### Python API version (no YAML)

In [9]:
import asyncio
from graph import Graph, Node, NodePolicy, GraphConfig, GraphExecutor
from neurosurfer.tools import Toolkit
from neurosurfer.models.chat_models.openai import OpenAIModel

# Reuse your existing toolkit + model
llm = LLM  # already created in your environment
tk = toolkit

graph = Graph(
    name="calc_and_explain",
    config=GraphConfig(max_concurrency=2),
    inputs_schema={"prompt": str},
    nodes=[
        Node(
            id="rewrite",
            fn="general_query_assistant",  # adjust name if needed
            inputs={
                # swap "query" -> "prompt" if your tool expects "prompt"
                "query": (
                    "You will receive a user request. Extract a SINGLE pure arithmetic expression that can be "
                    "evaluated by a calculator (e.g., '(42 * 7) - 5^2' or '0.035 * 12000').\n"
                    "- Do NOT include explanations.\n"
                    "- Return ONLY the expression as plain text.\n\n"
                    "User request:\n${inputs.prompt}"
                )
            },
            outputs=["num1", "num2", "operation"],
            policy=NodePolicy(
                retries=1,
                timeout_s=30,
                budget={"max_new_tokens": 128, "temperature": 0.1},
            ),
        ),
        Node(
            id="compute",
            fn="calculator",
            inputs={"num1": "${rewrite.num1}", "num2": "${rewrite.num2}", "operation": "${rewrite.operation}"},
            outputs=["text"],
            policy=NodePolicy(retries=0, timeout_s=15),
        ),
        Node(
            id="explain",
            fn="general_query_assistant",
            inputs={
                "query": (
                    "Original request: ${inputs.prompt}\n"
                    "Calculator result: ${compute.text}\n\n"
                    "Write a brief, user-friendly explanation of the result (one short paragraph)."
                )
            },
            outputs=["text"],
            policy=NodePolicy(
                retries=1,
                timeout_s=30,
                budget={"max_new_tokens": 180, "temperature": 0.2},
            ),
        ),
    ],
    outputs={"answer": "${explain.text}"},
)

executor = GraphExecutor(llm=llm, toolkit=tk, max_concurrency=2)

result = await run_async(
    executor.run(graph, inputs={"prompt": "Compute 3.5% of 12000 and explain"}, stream=True)
)

print("OK:", result.ok)
print("Answer:\n", result.outputs["answer"])


OK: True
Answer:
 The calculator result for your request is ${compute.text}. This means that after performing the calculation based on your input, the final answer is ${compute.text}. Let me know if you need further assistance!


### Planner-based path (using the YAML as a skeleton)

In [ ]:
import asyncio, tempfile, pathlib
from graph import PlannerAgent, FlowLoader, GraphExecutor

# 1) Write the YAML to a temp file (only for this demo)
yaml_text = r"""
name: calc_and_explain
inputs:
  prompt: str
config:
  max_concurrency: 2
nodes:
  - id: rewrite
    kind: task
    fn: general_query_assistant
    inputs:
      query: |
        You will receive a user request. Extract a SINGLE pure arithmetic expression that can be
        evaluated by a calculator (e.g., "(42 * 7) - 5^2" or "0.035 * 12000").
        - Do NOT include explanations.
        - Return ONLY the expression as plain text.

        User request:
        ${inputs.prompt}
    outputs: ["text"]
    policy: { retries: 1, timeout_s: 30, budget: { max_new_tokens: 128, temperature: 0.1 } }

  - id: compute
    kind: task
    fn: calculator
    inputs: { expression: ${rewrite.text} }
    outputs: ["text"]

  - id: explain
    kind: task
    fn: general_query_assistant
    inputs:
      query: |
        Original request: ${inputs.prompt}
        Calculator result: ${compute.text}

        Write a brief, user-friendly explanation of the result (one short paragraph).
    outputs: ["text"]
    policy: { retries: 1, timeout_s: 30, budget: { max_new_tokens: 180, temperature: 0.2 } }

outputs: { answer: ${explain.text} }
""".strip()

tmp = pathlib.Path(tempfile.gettempdir()) / "calc_and_explain.yml"
tmp.write_text(yaml_text)

# 2) Use the planner with a skeleton (so it returns your YAML-based Graph)
planner = PlannerAgent(llm=LLM)  # LLM not used when skeleton is set
graph = planner.plan_from_query(query="Compute 3.5% of 12000 and explain", skeleton=str(tmp))

# 3) Execute
executor = GraphExecutor(llm=LLM, toolkit=toolkit, max_concurrency=2)
result = asyncio.run(executor.run(graph, inputs={"prompt": "Compute 3.5% of 12000 and explain"}))

print("OK:", result.ok)
print(result.outputs["answer"])


Test ToolsRouterAgent

In [8]:
query = "Perform the calculation 20 * 90"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")


INFO     | 2025-11-06 11:08:32 | tools_router_agent.py:run | [router] Using tool: calculator
INFO     | 2025-11-06 11:08:32 | tools_router_agent.py:run | [router] Raw inputs: {'num1': 20.0, 'num2': 90.0, 'operation': 'multiply'}
1800.0

In [9]:
query = "Tell me a light-hearted joke!"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")

INFO     | 2025-11-06 11:08:33 | tools_router_agent.py:run | [router] Using tool: general_query_assistant
INFO     | 2025-11-06 11:08:33 | tools_router_agent.py:run | [router] Raw inputs: {'query': 'Tell me a light-hearted joke!'}
Why don't skeletons fight each other? They don't have the guts!None

## ReactAgent

In [21]:
from neurosurfer.agents.react import ReActAgent, ReActConfig

react_agent = ReActAgent(
    toolkit=toolkit,
    llm=LLM,
    specific_instructions="Always be concise in your answers. Break the task into steps if needed.",
    config=ReActConfig(
        temperature=0.7,
        max_new_tokens=4096,
        allow_input_pruning=True,
        repair_with_llm=True,
        skip_special_tokens=True,
        verbose=True
    )
)

# print(react_agent._system_prompt())
TASK = """Calculate 300 - 300. Then tell me a light-hearted joke about that result."""

for chunk in react_agent.run(TASK):
    print(chunk, end="")




[🧠] Chain of Thoughts...
Thought: I will first calculate 300 - 300 using the calculator tool, and then I will use the general_query_assistant tool to find a light-hearted joke about the result.

Action: {
  "tool": "calculator",
  "inputs": {
    "num1": 300,
    "num2": 300,
    "operation": "subtract"
  },
  "final_answer": false
}

[🔧] Tool: calculator
[📤] Inputs: {'num1': 300, 'num2': 300, 'operation': 'subtract'}

Observation:



[🧠] Chain of Thoughts...
Thought: The result of the calculation is 0. Now, I will use the general_query_assistant tool to find a light-hearted joke about the result.

Action: {
  "tool": "general_query_assistant",
  "inputs": {
    "query": "Tell me a light-hearted joke about the number 0."
  },
  "final_answer": true
}

[🔧] Tool: general_query_assistant
[📤] Inputs: {'query': 'Tell me a light-hearted joke about the number 0.'}

Why did the number 0 break up with the number 8?  
Because it found someone more "8" (8) than a zero!

Observation: Why did the number 0 break up with the number 8?  
Because it found someone more "8" (8) than a zero!



[🧠] Chain of Thoughts...
Thought: The calculation result is 0, and the joke provided is ready. The final answer is complete.

Final Answer: The result of 300 - 300 is 0. Here's a light-hearted joke about it: Why did the number 0 break up with the number 8? Because it found someone more "8" (8) than a zero!